<a href="https://colab.research.google.com/github/AnuBaluguri/AI/blob/main/ASS6_19K532.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***Importing Libraries***

In [ ]:
import pandas as pd
import numpy as np
import math as mt
import random

# ***Importing Data***

In [37]:
data = pd.read_excel("/content/drive/MyDrive/slr09.xls")
data.head()

*** No CODEPAGE record, no encoding_override: will use 'ascii'


,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171


# ***Outliers Detection***

In [38]:
outliers=[]
def outliers_detection(data):
  thres=3
  mean=np.mean(data)
  std=np.std(data)
  for i in data:
    z_score=(i-mean)/std
    if(np.abs(z_score)>thres):
      outliers.append(i)
  return outliers

In [8]:
outliers_x = outliers_detection(data['X'])
outliers_y=outliers_detection(data['Y'])
print("Outliers in column X(pH value): ",outliers_x)
print("Outliers in column Y(Bicarbonates): ",outliers_y)

Outliers in column X(pH value):  []
Outliers in column Y(Bicarbonates):  []


# ***Outliers Treatment***

In [39]:
from scipy.stats import zscore
zx=np.abs(zscore(data['X']))
zy=np.abs(zscore(data['Y']))
data['z_score_x']=zx
data['z_score_y']=zy
outliersx=len(data[data['z_score_x']>3])
outliersy=len(data[data['z_score_y']>3])
data_z = data[data['z_score_x']<3]
data=data_z.drop(['z_score_x', 'z_score_y'], axis=1)
data

,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171
5,7.8,143
6,7.3,217
7,8.0,190
8,7.1,142
9,7.5,190


# ***Missing Value Treatment***

In [14]:
print(data.isnull().sum())

X    0
Y    0
dtype: int64


In [ ]:
m1=data['X'].mean
m2=data['Y'].mean
data['X'].fillna(value=m1,inplace=True)
data['Y'].fillna(value=m2,inplace=True)
data

# ***Data Splitting***

In [19]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data.X,data.Y,test_size=0.50,random_state=10)


# ***Intialization***

In [20]:
m1=random.uniform(-1,1)
m2=random.uniform(-1,1)
c=random.uniform(-1,1)
eta=0.0001
max_iter=10

# ***Gradient Calculation***

In [29]:
def gradient(m1,m2,c,x,y):
  m1_deriv=-1*(y-m1*x*x-m2*x-c)*x*x
  m2_deriv=-1*(y-m1*x*x-m2*x-c)*x
  c_deriv=-1*(y-m1*x*x-m2*x-c)
  return m1_deriv,m2_deriv,c_deriv

# ***Training Using Stochastic Gradient Descent Optimizer***

In [40]:
for iter in range(max_iter):
  for sample in range(x_train.shape[0]):
    del_m1,del_m2,del_c=gradient(m1, m2, c, x_train.iloc[sample], y_train.iloc[sample])
    m1=m1-(eta)*del_m1
    m2=m2-(eta)*del_m2
    c=c-(eta)*del_c
print("Model Parameters m1, m2 and c : ",m1,m2,c)

Model Parameters m1, m2 and c :  2.37756111212204 0.48441521776662666 0.6804119799960171


# ***Model Prediction On Training Data***

In [32]:
y_pred_train=[]
for i in x_train:
  y_pr_tr=(m1*i*i)+m2*i+c
  y_pred_train.append(y_pr_tr)
y_pred_train=np.array(y_pred_train)

# ***Model Prediction On Testing Data***

In [33]:
y_pred_test=[]
for i in x_test:
  y_pr_ts=(m1*i*i)+m2*i+c
  y_pred_test.append(y_pr_ts)
y_pred_train=np.array(y_pred_test)

# ***Error Metrics Calculation***

In [34]:
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
rmse=math.sqrt(mean_squared_error(y_train,y_pred_train))
print("Root mean square error: ",rmse)
mse=mean_squared_error(y_train,y_pred_train)
print("Mean square error: ",mse)
mae=mean_absolute_error(y_train,y_pred_train)
print("Mean absolute error: ",mae)

Root mean square error:  60.86793682138679
Mean square error:  3704.905732892334
Mean absolute error:  50.34278072733047


In [35]:
#testing
rmse=math.sqrt(mean_squared_error(y_test,y_pred_test))
print("Root mean square error: ",rmse)
mse=mean_squared_error(y_test,y_pred_test)
print("Mean square error: ",mse)
mae=mean_absolute_error(y_test,y_pred_test)
print("Mean absolute error: ",mae)

Root mean square error:  65.34501190564629
Mean square error:  4269.970580949056
Mean absolute error:  52.215210058327145


# ***Deployment***

In [36]:
x=input("Enter pH value of well water: ")
bicarbonates=m1*float(x)*float(x)+m2*float(x)+c
print("Bicarbonates of well water: ",bicarbonates)

Enter pH value of well water: 5
Bicarbonates of well water:  61.96028698208825
